In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
iris_data = pd.read_csv('Iris.csv')

iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
iris_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [4]:
def map_strings_to_numbers(string_list):
    unique_strings = list(set(string_list))
    string_to_number = {string: number for number, string in enumerate(unique_strings)}

    # Ersetze die Strings durch numerische Werte
    numerical_list = [string_to_number[string] for string in string_list]

    return numerical_list, string_to_number

In [5]:
# Definiere die Spaltennamen
features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
lables = 'Species'

# Aufteilen der Daten in Features (X) und Labels (y)
X = iris_data[features].values
y = iris_data[lables].values

# Transformation zu Nummern
y, string_to_number_dict = map_strings_to_numbers(y)

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4, random_state=42)

#Aufteilen der X_Test und y_test Daten
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [6]:
# Normalisieren der Daten
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.transform(X_test)

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class YourDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Erstellen von Trainings- und Testdatensätzen
train_dataset = YourDataset(X_train, y_train)
val_dataset = YourDataset(X_val, y_val)
test_dataset = YourDataset(X_test, y_test)

# Erstellen von Datenladern
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Definieren des Modells, der Verlustfunktion und des Optimierers
input_size = 4  # Anzahl der Merkmale im Iris-Datensatz
hidden_size = 8 # Anzahl verstecket Neuronen
num_classes = 10  # Anzahl der Klassen 
model = SimpleNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
num_epochs = 0
accuracy = 0

while accuracy != 1.0:
    num_epochs += 1
    for inputs, labels in train_loader:
        # Vorwärtsdurchlauf
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Rückwärtsdurchlauf und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {num_epochs}, Loss: {loss.item()}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Val Accuracy: {accuracy * 100:.2f}%')

Epoch 1, Loss: 2.2629177570343018
Val Accuracy: 0.00%
Epoch 2, Loss: 2.2418274879455566
Val Accuracy: 0.00%
Epoch 3, Loss: 2.2178642749786377
Val Accuracy: 0.00%
Epoch 4, Loss: 2.2434558868408203
Val Accuracy: 3.33%
Epoch 5, Loss: 2.21327543258667
Val Accuracy: 3.33%
Epoch 6, Loss: 2.0582501888275146
Val Accuracy: 6.67%
Epoch 7, Loss: 2.0848426818847656
Val Accuracy: 6.67%
Epoch 8, Loss: 2.076110363006592
Val Accuracy: 10.00%
Epoch 9, Loss: 1.8887815475463867
Val Accuracy: 13.33%
Epoch 10, Loss: 1.943354845046997
Val Accuracy: 33.33%
Epoch 11, Loss: 1.7898261547088623
Val Accuracy: 33.33%
Epoch 12, Loss: 1.878023386001587
Val Accuracy: 40.00%
Epoch 13, Loss: 1.738572359085083
Val Accuracy: 60.00%
Epoch 14, Loss: 1.717690110206604
Val Accuracy: 76.67%
Epoch 15, Loss: 1.774990439414978
Val Accuracy: 83.33%
Epoch 16, Loss: 1.7006133794784546
Val Accuracy: 86.67%
Epoch 17, Loss: 1.3864047527313232
Val Accuracy: 86.67%
Epoch 18, Loss: 1.4271275997161865
Val Accuracy: 86.67%
Epoch 19, Loss: 

In [28]:
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

def map_numbers_to_strings(numerical_list, string_to_number_dict):
    # Umkehren des Mappings
    number_to_string_dict = {number: string for string, number in string_to_number_dict.items()}

    # Ersetze numerische Werte durch Strings
    original_strings = [number_to_string_dict[number] for number in numerical_list]

    return original_strings


map_numbers_to_strings(predicted.tolist(), string_to_number_dict)

Test Accuracy: 96.83%


['Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa']